<a href="https://colab.research.google.com/github/gmauricio-toledo/NLP-MCD/blob/main/10-Deteccion_Evasi%C3%B3n_Lexica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Detectando Evasión Léxica con Embeddings de Palabras</h1>

En esta notebook exploramos cómo los embeddings de palabras, como los generados por Word2Vec o FastText, pueden utilizarse para detectar intentos de evasión léxica en textos, un fenómeno común en redes sociales donde se alteran palabras ofensivas o prohibidas mediante sustituciones de caracteres (por ejemplo, "m4t4r" en lugar de "matar") o el uso de sinónimos engañosos (como "desvivir"). Entrenamos un modelo de embeddings a partir de un corpus de subtítulos en español modificado artificialmente para incluir estas variantes, y evaluamos la capacidad del modelo para capturar la cercanía semántica entre las palabras originales y sus versiones disfrazadas mediante similitud coseno.


Este enfoque permite ir más allá de los filtros basados únicamente en coincidencias exactas, ofreciendo una estrategia más robusta para la moderación de contenido, detección de spam, prevención de acoso en línea y otras aplicaciones donde el lenguaje se adapta intencionalmente para eludir sistemas de control.

In [ ]:
!pip install gensim

# Corpus original

El corpus original se encuentra en [HuggingFace](https://huggingface.co/datasets/Helsinki-NLP/opus-100). `OPUS-100` es un corpus multilingüe centrado en inglés que abarca 100 idiomas. Al ser centrado en inglés, todos los pares de entrenamiento incluyen inglés ya sea en el lado de origen o en el de destino. Los idiomas se seleccionaron según la cantidad de datos paralelos disponibles en OPUS. El corpus contiene aproximadamente 55 millones de pares de oraciones. De los 99 pares de idiomas, 44 tienen 1 millón de pares de oraciones de datos de entrenamiento, 73 tienen al menos 100 mil y 95 tienen al menos 10 mil. La tarea principal que soporta es la traducción automática.

In [ ]:
from datasets import load_dataset

dataset = load_dataset("Helsinki-NLP/opus-100", "en-es")

In [ ]:
# Extraer solo el texto en español
textos_espanol = []

for ejemplo in dataset['train']:
    texto_es = ejemplo['translation']['es']
    textos_espanol.append(texto_es)

# Algunos ejemplos
print(textos_espanol[:5])
print(f"\nTotal de oraciones: {len(textos_espanol)}")

# Versión preprocesada

In [ ]:
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize

In [ ]:
!gdown 1CkJEbUJOB6u83_r3jhGq_KTh4NRRices

In [ ]:
import pandas as pd

df = pd.read_csv('opus100_subtitulos_esp.csv')
df

In [ ]:
word = 'matar'

target_docs_idxs = []

for idx in df.index.to_list():
    doc = df.loc[idx,'texto']
    if word in word_tokenize(doc):
        target_docs_idxs.append(idx)

In [ ]:
import numpy as np

word = 'matar'
replacing_words = ['m4t4r','desvivir']
idx_to_replace = np.random.choice(target_docs_idxs,size=300,replace=False)
# docs = list(df['texto sin numeros'].values)

for idx in idx_to_replace:
    doc = df.loc[idx,'texto']
    new_word = np.random.choice(replacing_words,size=1)[0]
    doc = doc.replace(word,new_word)
    df.loc[idx,'texto'] = doc
    # docs.append(doc)

In [ ]:
training_docs = list(df['texto'].values)
training_docs = [word_tokenize(doc) for doc in training_docs]

In [ ]:
from gensim.models import Word2Vec

model = Word2Vec(training_docs, vector_size=300, window=5, min_count=2, workers=4)

In [ ]:
model.wv.most_similar('matar')

Comparemos la similitud de estas alteraciones léxicas con respecto a la original

In [ ]:
import matplotlib.pyplot as plt

mas_similares = model.wv.most_similar('matar',topn=200)
similaridades = [similaridad for _, similaridad in mas_similares]

plt.figure()
plt.hist(similaridades)
plt.show()

## Aplicaciones

* Moderación de contenido en redes sociales y foros.
* Detección de spam o phishing: identificar mensajes maliciosos que evitan palabras clave bloqueadas (ej. “gr4tis”, “c1ick aquí”).
* Prevención de acoso en línea: detectar insultos camuflados mediante alteraciones ortográficas o uso de emojis/símbolos.
* Identificación de jerga en contextos específicos: por ejemplo, en comunidades online donde se crean nuevas formas para evitar censura.
* Mejora de sistemas de autocorrección o normalización de texto: mapear variantes informales o erróneas a sus formas canónicas usando vecindad semántica.